# Extração e Classificação de BI-RADS em Laudos de Mamografia do Pardini

## Introdução Técnica

Este notebook implementa um sistema automatizado para extração, classificação e processamento de informações de BI-RADS (Breast Imaging Reporting and Data System) a partir de laudos de mamografia do laboratório Pardini. O sistema identifica pacientes com exames em atraso e gera listas de ativação para envio de notificações, com base na classificação BI-RADS e no tempo recomendado para realização de novo exame.

### Objetivo Principal

O objetivo principal deste notebook é extrair automaticamente as classificações BI-RADS (0-6) de laudos de mamografia do Pardini, calcular datas de retorno recomendadas com base nessa classificação, e identificar pacientes elegíveis para notificações de acompanhamento. Pacientes com BI-RADS 1, 2 e 3 são incluídos em uma lista de ativação para monitoramento e notificações de retorno.

### Tecnologias Utilizadas

- **Framework de Processamento**: Apache Spark (PySpark)
- **Armazenamento de Dados**: Delta Lake
- **Manipulação de Dados**: PySpark SQL e DataFrame API
- **Processamento de Texto**: Expressões Regulares (RegEx)
- **Monitoramento e Alertas**: Octoops (Sentinel)
- **Logging**: Biblioteca padrão Python (logging)

### Fluxo de Trabalho/Etapas Principais

1. **Configuração do Ambiente**: Instalação de dependências e configuração de logging
2. **Definição de Parâmetros**: Configuração das tabelas e filtros para consultas SQL
3. **Extração de BI-RADS**: Consulta SQL complexa que:
   - Utiliza RegEx para extrair classificações BI-RADS dos laudos
   - Transforma códigos romanos em valores numéricos (I→1, II→2, etc.)
   - Calcula valores mínimo, máximo e atual de BI-RADS
4. **Processamento de Datas de Retorno**: Cálculo das datas previstas de retorno com base na classificação BI-RADS
5. **Filtragem e Transformação**: Aplicação de transformações nos dados e filtragem de pacientes elegíveis
6. **Persistência de Dados**: Salvamento dos dados processados em tabelas Delta

### Dados Envolvidos

- **Fonte de Dados**: 
  - Tabela: `refined.saude_preventiva.pardini_laudos`
  - Filtro principal: `linha_cuidado = 'mama'`
  - Tipos de exame: 'MAMO', 'MAMODI'

- **Tabelas de Destino**:
  - `refined.saude_preventiva.pardini_laudos_mama_birads`: Dados gerais dos laudos com classificação BI-RADS
  - `refined.saude_preventiva.pardini_laudos_mama_birads_ativacao`: Subconjunto com pacientes elegíveis para notificações

- **Colunas Principais**:
  - `ficha`, `sigla_exame`, `id_marca`, `sequencial`: Identificadores do exame
  - `MIN_BIRADS`, `MAX_BIRADS`, `BIRADS`: Valores de classificação BI-RADS
  - `dth_pedido`: Data do pedido do exame
  - `dth_previsao_retorno`: Data calculada para retorno do paciente
  - `retorno_cliente`: Tempo recomendado para retorno em meses (12 para BI-RADS 1-2, 6 para BI-RADS 3)

### Resultados/Saídas Esperadas

1. **Tabela Estruturada de BI-RADS**: Tabela Delta contendo todos os exames de mamografia do Pardini com classificação BI-RADS extraída
2. **Lista de Ativação**: Tabela Delta contendo apenas pacientes:
   - Com BI-RADS 1, 2 ou 3
   - Sem retorno registrado
   - Do sexo feminino entre 40-75 anos

### Pré-requisitos

- **Ambiente Databricks**: Com acesso ao Delta Lake e capacidade de execução de PySpark
- **Dependência Principal**: Pacote `octoops` para monitoramento e alertas
- **Acesso a Dados**: Permissões de leitura na tabela `refined.saude_preventiva.pardini_laudos`
- **Permissões de Escrita**: Para criar ou atualizar as tabelas de destino

### Considerações Importantes

- **Extração via RegEx**: A extração de BI-RADS usa expressões regulares sofisticadas para identificar padrões como "BIRADS X", "CATEGORIA X" ou notações em algarismos romanos
- **Recomendações de Retorno**: Seguem o protocolo médico padrão:
  - BI-RADS 1-2: Retorno em 12 meses (360 dias)
  - BI-RADS 3: Retorno em 6 meses (180 dias)
  - BI-RADS 4-6: Sem recomendação automática de retorno (requer avaliação médica)
- **Deduplicação**: A tabela de ativação remove duplicatas por `ficha` para evitar múltiplas notificações para o mesmo paciente
- **Tratamento de Erros**: Sistema robusto com captura de exceções e notificações via Sentinel em caso de falhas

## Instalação da Biblioteca Octoops

Esta célula realiza a instalação da biblioteca `octoops` utilizando o comando `pip` com a sintaxe especial de magic command do Jupyter (`%pip`). 

A biblioteca Octoops é uma ferramenta interna utilizada para monitoramento e alertas em pipelines de dados. Ela fornece funcionalidades como:

- Envio de notificações em caso de falha no processamento
- Monitoramento de execuções de pipelines
- Registro centralizado de eventos e métricas

A instalação ocorre no ambiente atual de execução e é essencial para o funcionamento do sistema de alertas implementado nas células posteriores deste notebook.

In [0]:
%pip install octoops

## Reinicialização do Ambiente Python

Esta célula reinicia o interpretador Python no ambiente Databricks através da função `dbutils.library.restartPython()`. Esta etapa é necessária após a instalação de pacotes via pip para garantir que as novas bibliotecas sejam carregadas corretamente.

A reinicialização:
- Finaliza a sessão Python atual
- Inicia uma nova sessão com todas as bibliotecas recém-instaladas disponíveis
- Mantém o contexto do notebook, permitindo que células anteriores permaneçam executadas
- Garante que a biblioteca Octoops estará disponível para uso nas células seguintes

Esta operação é equivalente a reiniciar o kernel no Jupyter Notebook tradicional, mas de forma mais controlada no ambiente Databricks.

In [0]:
dbutils.library.restartPython()

## Importação de Bibliotecas e Módulos

Esta célula importa todas as bibliotecas e módulos necessários para o processamento dos dados, manipulação de DataFrames, operações com datas, logging e envio de alertas.

### Bibliotecas importadas:

#### PySpark SQL e Funções
- **Funções de coluna e manipulação de dados**: `col`, `year`, `month`, `dayofmonth`, `when`, `lit`, `expr`, `to_timestamp`
- **Tipos de dados**: `DateType`
- **DataFrame API**: `DataFrame` (para tipagem)
- **Funções de data**: `datediff`, `to_date`

#### Bibliotecas Padrão Python
- **Manipulação de datas**: `datetime`
- **Logging**: `logging`
- **Interação com sistema**: `sys`
- **Tratamento de erros**: `traceback`

#### Monitoramento (Octoops)
- `OctoOps`: Framework principal para monitoramento
- `Sentinel`: Componente específico para envio de alertas

Estas importações fornecem as ferramentas necessárias para executar o pipeline completo de extração, processamento e persistência de dados de BI-RADS dos laudos do Pardini.

In [0]:
from pyspark.sql.functions import col, year, month, dayofmonth, when, lit, expr, to_timestamp
from pyspark.sql.types import DateType
from pyspark.sql import DataFrame
from pyspark.sql.functions import datediff, to_date
from datetime import datetime
import logging
import sys
import traceback
from octoops import OctoOps
from octoops import Sentinel

## Configuração do Logger

Esta célula inicializa um objeto logger para registrar eventos e mensagens durante a execução do notebook. O logger é configurado utilizando a biblioteca padrão de logging do Python.

A configuração `logging.getLogger(__name__)` cria um logger associado ao nome do módulo atual (neste caso, o notebook em execução). Esta é uma prática recomendada pois permite:

1. Identificação clara da origem das mensagens de log
2. Configuração hierárquica de loggers quando necessário
3. Controle granular dos níveis de logging

Este logger será utilizado posteriormente no código para registrar informações importantes, como avisos sobre DataFrames vazios ou detalhes sobre operações de persistência de dados, auxiliando na depuração e monitoramento do pipeline.

In [0]:
logger = logging.getLogger(__name__)

## Configuração de Tabelas e Filtros

Esta célula define as tabelas de destino para os dados processados e configura os filtros SQL que serão aplicados nas consultas. A configuração é crucial para o processamento incremental e para a definição dos critérios de elegibilidade para notificações.

### Tabelas Definidas

- **`table_birads`**: Tabela principal para armazenar todos os laudos processados com classificação BI-RADS
  ```python
  table_birads = "refined.saude_preventiva.pardini_laudos_mama_birads"
  ```

- **`table_birads_ativacao`**: Tabela para armazenar apenas os pacientes elegíveis para notificações
  ```python
  table_birads_ativacao = "refined.saude_preventiva.pardini_laudos_mama_birads_ativacao"
  ```

### Filtros Configurados

1. **`where_clause`**: Filtro para processamento incremental
   - Inicialmente vazio
   - Se a tabela principal já existir, configura-se para processar apenas registros mais recentes que o último processamento
   - Utiliza subconsulta para determinar o valor de `_datestamp` mais recente na tabela existente

2. **`filtro_ativacao`**: Filtro para selecionar pacientes elegíveis para notificações
   - Pacientes sem retorno registrado (`eleg.ficha IS NULL`)
   - Com BI-RADS 1, 2 ou 3 (`brd.BIRADS IN (1, 2, 3)`)
   - Que realizaram exames específicos (`flr.sigla_exame IN ('MAMODI','MAMO')`)
   - Do sexo feminino (`UPPER(flr.sexo_cliente) = 'F'`)
   - Na faixa etária apropriada (40-75 anos)

### Impacto

Estas configurações otimizam o processamento, evitando reprocessar dados antigos e focando nas notificações apenas para os pacientes que realmente precisam ser acompanhados de acordo com os protocolos médicos para cada classificação BI-RADS.

In [0]:
table_birads = "refined.saude_preventiva.pardini_laudos_mama_birads"
table_birads_ativacao = "refined.saude_preventiva.pardini_laudos_mama_birads_ativacao"
where_clause = ""
 
if spark.catalog.tableExists(table_birads):
    where_clause = f"""
    WHERE        
        flr._datestamp > (
            SELECT MAX(brd._datestamp)
            FROM {table_birads} brd
        )
        
    """
 
filtro_ativacao = """
    WHERE
        eleg.ficha IS NULL
        AND brd.BIRADS IN (1, 2, 3)
        AND flr.sigla_exame IN ('MAMODI','MAMO')
        AND UPPER(flr.sexo_cliente) = 'F'
        AND (
            idade_cliente >= 40 AND idade_cliente < 76
        )
        AND flr.marca = 'HERMES PARDINI (MG)'
"""

## Consulta SQL para Extração de BI-RADS

Esta célula implementa uma consulta SQL complexa para extrair classificações BI-RADS de laudos de mamografia do Pardini e criar dois DataFrames: um com todos os laudos processados e outro apenas com pacientes elegíveis para notificações.

### Objetivo da Célula

A célula realiza o processamento principal do notebook:
1. Define uma consulta SQL que extrai e processa informações de BI-RADS
2. Cria dois DataFrames a partir desta consulta com diferentes filtros

### Estrutura da Consulta

A consulta utiliza Common Table Expressions (CTEs) para organizar o processamento em etapas lógicas:

1. **CTE `base`**: 
   - Extrai os identificadores básicos dos exames (`ficha`, `sigla_exame`, `id_marca`, `sequencial`)
   - Captura o texto do laudo
   - Usa expressões regulares sofisticadas para extrair menções a BI-RADS
   - Transforma algarismos romanos (I, II, III, etc.) em valores numéricos
   - Filtra laudos da linha de cuidado 'mama' e tipos específicos de exame

2. **CTE `dados_birads`**: 
   - Calcula métricas agregadas de BI-RADS:
     - `MIN_BIRADS`: Menor valor de BI-RADS encontrado no laudo
     - `MAX_BIRADS`: Maior valor de BI-RADS encontrado no laudo
     - `BIRADS`: Valor mais recente/final (último elemento do array)

3. **CTE `dados_laudos`**: 
   - Coleta dados demográficos e administrativos do paciente
   - Calcula a idade do cliente a partir da data de nascimento
   - Aplica o filtro incremental (`where_clause`)

4. **Consulta Final**: 
   - Combina os dados das CTEs através de JOINs
   - Une com a tabela de retornos elegíveis (`pardini_retorno_elegivel_ficha`)
   - Seleciona todas as colunas relevantes para análise
   - Aplica o filtro de ativação quando apropriado

### Expressões Regulares

A extração do BI-RADS utiliza expressões regulares complexas:

```python
REGEXP_EXTRACT_ALL(
    REGEXP_EXTRACT(
        REGEXP_REPLACE(UPPER(flr.laudo_tratado), r'[-:®]|\xa0', ''),
        r'(?mi)(AVALIA[CÇ][AÃ]O|CONCLUS[AÃ]O|IMPRESS[AÃ]O|OPINI[AÃ]O)?(.*)', 2
    ),
    r"(?mi)(BIRADS|CATEGORI[AO]|CATEGORA|CATEGORIA R|CAT\W)\s*(\d+\w*|VI|V|IV|III|II|I)\W*\w?(BIRADS|CATEGORI[AO]|CATEGORA|CATEGORIA R)?(\W|$)", 2
)
```

Esta expressão:
1. Normaliza o texto (maiúsculas, remoção de caracteres especiais)
2. Extrai a seção relevante do laudo (após conclusão/impressão diagnóstica)
3. Localiza menções a BI-RADS em diversos formatos
4. Captura o valor numérico ou romano da classificação

### DataFrames Criados

A célula cria dois DataFrames importantes:

1. **`df_spk`**: DataFrame principal contendo todos os laudos processados
   ```python
   df_spk = spark.sql(query.format(where_clause = where_clause, filtro_ativacao = ""))
   ```

2. **`df_spk_ativacao`**: DataFrame filtrado apenas com pacientes elegíveis para notificações
   ```python
   df_spk_ativacao = spark.sql(query.format(where_clause = "", filtro_ativacao = filtro_ativacao))
   ```

In [0]:
query = """
WITH base AS (
    SELECT
        flr.ficha,
        flr.sigla_exame, 
        flr.id_marca,
        flr.sequencial,
        flr.laudo_tratado,
        REGEXP_EXTRACT_ALL(
            REGEXP_EXTRACT(
                REGEXP_REPLACE(UPPER(flr.laudo_tratado), r'[-:®]|\xa0', ''),
                r'(?mi)(AVALIA[CÇ][AÃ]O|CONCLUS[AÃ]O|IMPRESS[AÃ]O|OPINI[AÃ]O)?(.*)', 2
            ),
            r"(?mi)(BIRADS|CATEGORI[AO]|CATEGORA|CATEGORIA R|CAT\W)\s*(\d+\w*|VI|V|IV|III|II|I)\W*\w?(BIRADS|CATEGORI[AO]|CATEGORA|CATEGORIA R)?(\W|$)", 2
        )        
        AS RAW_BIRADS,
        FILTER(
            TRANSFORM(RAW_BIRADS, x ->
                CASE
                    WHEN x = "I" THEN 1
                    WHEN x = "II" THEN 2
                    WHEN x = "III" THEN 3
                    WHEN x = "IV" THEN 4
                    WHEN x = "V" THEN 5
                    WHEN x = "VI" THEN 6
                    WHEN TRY_CAST(x AS INT) > 6 THEN NULL
                    ELSE REGEXP_REPLACE(x, r'[^0-9]', '')
                END
            ), x -> x IS NOT NULL
        ) AS CAT_BIRADS
    FROM refined.saude_preventiva.pardini_laudos flr
    WHERE
        flr.linha_cuidado = 'mama'
        AND UPPER(flr.sexo_cliente) = 'F' 
        AND flr.sigla_exame IN ('MAMO','MAMODI')
),
 
dados_birads AS (
    SELECT
        *,
        ARRAY_MIN(CAT_BIRADS) AS MIN_BIRADS,
        ARRAY_MAX(CAT_BIRADS) AS MAX_BIRADS,
        TRY_ELEMENT_AT(CAST(CAT_BIRADS AS ARRAY<INT>), -1) AS BIRADS
    FROM base
),
 
dados_laudos AS (
    SELECT
        flr.linha_cuidado,
        flr.id_unidade,
        flr.id_ficha,
        flr.id_exame, 
        flr.id_marca,
        flr.sequencial,
        flr.ficha,
        flr.id_cliente,
        flr.pefi_cliente,
        flr.sigla_exame,
        flr.marca,
        flr.laudo_tratado,
        (
          TIMESTAMPDIFF(DAY, flr.dth_nascimento_cliente, CURDATE()) / 365.25
        ) AS idade_cliente,
        flr.sexo_cliente,
        flr.dth_pedido,
        flr._datestamp
    FROM refined.saude_preventiva.pardini_laudos flr
    {where_clause}
)
 
SELECT
    flr.* except(idade_cliente),
    brd.MIN_BIRADS,
    brd.MAX_BIRADS,
    brd.BIRADS,
 
    eleg.dth_pedido        AS dth_pedido_retorno_elegivel,
    eleg.ficha             AS ficha_retorno_elegivel,
    eleg.siglas_ficha      AS siglas_ficha_retorno_elegivel,
    eleg.marca             AS marca_retorno_elegivel,
    eleg.unidade           AS unidade_retorno_elegivel,
    eleg.convenio          AS convenio_retorno_elegivel,
    eleg.valores_exame     AS valores_exame_retorno_elegivel,
    eleg.valores_ficha     AS valores_ficha_retorno_elegivel,
    eleg.qtd_exame         AS qtd_exame_retorno_elegivel,
    eleg.secao             AS secao_retorno_elegivel,
    eleg.dias_entre_ficha  AS dias_entre_ficha_elegivel

FROM dados_laudos flr
INNER JOIN dados_birads brd
    ON flr.ficha = brd.ficha
    AND flr.sigla_exame = brd.sigla_exame    
    AND flr.id_marca = brd.id_marca
    AND flr.sequencial = brd.sequencial    
LEFT JOIN refined.saude_preventiva.pardini_retorno_elegivel_ficha eleg
    ON eleg.ficha_origem = flr.ficha
    AND eleg.id_cliente = flr.id_cliente
    AND eleg.linha_cuidado = flr.linha_cuidado
{filtro_ativacao}
"""
 
df_spk = spark.sql(query.format(
    where_clause = where_clause,
    filtro_ativacao = ""
    )
)
 
df_spk_ativacao = spark.sql(query.format(
    where_clause = "",
    filtro_ativacao = filtro_ativacao
    )
)

## Função para Cálculo da Data Prevista de Retorno

Esta célula define a função `calcular_data_prevista`, que adiciona uma coluna ao DataFrame com a data recomendada de retorno para cada paciente, com base na sua classificação BI-RADS.

### Objetivo da Função

Calcular e adicionar a coluna `dth_previsao_retorno` ao DataFrame, definindo uma data futura recomendada para o próximo exame de mamografia, seguindo os protocolos médicos para acompanhamento de cada classificação BI-RADS.

### Parâmetros e Retorno

**Parâmetros:**
- `df_spk` (DataFrame): DataFrame Spark contendo a coluna `BIRADS` e `dth_pedido`

**Retorno:**
- DataFrame com a coluna adicional `dth_previsao_retorno`

### Lógica de Cálculo

A função utiliza expressões condicionais baseadas no valor de BI-RADS:

1. **BI-RADS 1 ou 2** (achados negativos ou benignos):
   - Adiciona 360 dias (aproximadamente 1 ano) à data do pedido
   - `when(col('BIRADS').isin([1, 2]), expr("date_add(dth_pedido, 360)"))`

2. **BI-RADS 3** (achados provavelmente benignos):
   - Adiciona 180 dias (aproximadamente 6 meses) à data do pedido
   - `when(col('BIRADS') == 3, expr("date_add(dth_pedido, 180)"))`

3. **Outros valores de BI-RADS** (4, 5, 6):
   - Define como None (null), pois requerem outros tipos de acompanhamento
   - `.otherwise(None)`

### Implementação Técnica

A implementação utiliza a API DataFrame do PySpark:
- `withColumn()` para adicionar uma nova coluna ao DataFrame
- `when()...otherwise()` para expressões condicionais
- `expr()` para aplicar a função SQL `date_add()` que realiza os cálculos de data

Esta abordagem segue os protocolos médicos padrão para acompanhamento de exames de mamografia baseado na classificação BI-RADS, onde categorias 1-2 são de baixo risco e categoria 3 requer acompanhamento mais frequente.

In [0]:
def calcular_data_prevista(df_spk: DataFrame):
    """
    Adiciona uma coluna 'dth_previsao_retorno' ao DataFrame com base na coluna 'BIRADS'.

    - Para BIRADS 1 ou 2, adiciona 360 dias à data da coluna 'dth_pedido'.
    - Para BIRADS 3, adiciona 180 dias à data da coluna 'dth_pedido'.
    - Para outros valores de BIRADS, define 'dth_previsao_retorno' como None.

    Parâmetros:
    df_spk (DataFrame): O DataFrame Spark contendo os dados de entrada.

    Retorna:
    DataFrame: O DataFrame atualizado com a nova coluna 'dth_previsao_retorno'.
    """
    df_spk = df_spk.withColumn(
        'dth_previsao_retorno',
        when(
            col('BIRADS').isin([1, 2]),
            expr("date_add(dth_pedido, 360)")
        ).when(
            col('BIRADS') == 3,
            expr("date_add(dth_pedido, 180)")  
        ).otherwise(None)
    )
    return df_spk

## Função de Transformação de Campos

Esta célula define a função `transform_fields`, responsável por aplicar várias transformações ao DataFrame para enriquecê-lo com informações calculadas e preparar os dados para análise posterior.

### Objetivo da Função

Aplicar um conjunto de transformações aos DataFrames gerados pela consulta SQL, adicionando novas colunas com informações derivadas e convertendo formatos de dados para tipos apropriados.

### Parâmetros e Retorno

**Parâmetros:**
- `df_spk` (DataFrame): DataFrame Spark a ser transformado

**Retorno:**
- DataFrame enriquecido com novas colunas e transformações

### Lógica Detalhada

A função executa as seguintes transformações, em sequência:

1. **Verificação de DataFrame Vazio**:
   - Verifica se o DataFrame está vazio usando `.isEmpty()`
   - Se vazio, registra um aviso no log e retorna o DataFrame sem alterações
   
2. **Adição da Coluna de Recomendação de Retorno**:
   - Cria a coluna `retorno_cliente` que indica o tempo recomendado para retorno em meses
   - BI-RADS 1-2: 12 meses
   - BI-RADS 3: 6 meses
   - Outros valores: 0 (sem recomendação automática)

3. **Cálculo da Data Prevista de Retorno**:
   - Chama a função auxiliar `calcular_data_prevista()` definida anteriormente
   
4. **Conversão de Formatos de Data**:
   - Converte `dth_pedido_retorno_elegivel` para formato timestamp
   - Converte `dth_previsao_retorno` para formato timestamp
   
5. **Cálculo do Intervalo entre Datas**:
   - Calcula a diferença em dias entre a data prevista de retorno e a data do pedido
   - Armazena o resultado na coluna `dias_ate_retorno`

### Variáveis Criadas/Modificadas

- **`retorno_cliente`**: Tempo de retorno recomendado em meses (12, 6 ou 0)
- **`dth_previsao_retorno`**: Data calculada para o próximo exame
- **`dias_ate_retorno`**: Número de dias entre o exame atual e o próximo recomendado

### Impacto

Esta função enriquece os dados com informações cruciais para o sistema de notificações, permitindo identificar quando um paciente deve ser contatado para realizar um novo exame de acompanhamento.

In [0]:
def transform_fields(df_spk: DataFrame) -> DataFrame:
  """
  Transforma os campos do DataFrame fornecido.

  - Verifica se o DataFrame está vazio. Se estiver, registra um aviso e retorna o DataFrame sem alterações.
  - Adiciona uma coluna 'retorno_cliente' com valores baseados na coluna 'BIRADS':
    - 12 meses para BIRADS 1 ou 2
    - 6 meses para BIRADS 3
    - 0 para outros valores
  - Calcula a data prevista de retorno usando a função 'calcular_data_prevista'.
  - Converte a coluna 'dth_pedido_retorno_elegivel' para o tipo timestamp.
  - Converte a coluna 'dth_previsao_retorno' para o tipo timestamp.
  - Calcula a diferença em dias entre 'dth_previsao_retorno' e 'dth_pedido', armazenando o resultado na coluna 'dias_ate_retorno'.

  Parâmetros:
  df_spk (DataFrame): O DataFrame a ser transformado.

  Retorna:
  DataFrame: O DataFrame transformado com as novas colunas.
  """

  if df_spk.isEmpty():
      logger.warning("No Data Found!")
      return df_spk

  df_spk = df_spk.withColumn(
      'retorno_cliente',
      when(col('BIRADS').isin([1, 2]), 12).when(col('BIRADS') == 3, 6).otherwise(0)
  )

  df_spk = calcular_data_prevista(df_spk)
  df_spk = df_spk.withColumn('dth_pedido_retorno_elegivel', to_timestamp(col('dth_pedido_retorno_elegivel')))
  df_spk = df_spk.withColumn('dth_previsao_retorno', to_timestamp(col('dth_previsao_retorno')))
  df_spk = df_spk.withColumn('dias_ate_retorno', datediff(to_date(col('dth_previsao_retorno')), to_date(col('dth_pedido'))))
  return df_spk

## Configuração de Webhook e Sistema de Tratamento de Erros

Esta célula configura o sistema de alerta e define a função `error_message` para gerenciar erros de forma padronizada, garantindo que falhas durante o processamento sejam devidamente registradas e comunicadas.

### Configurações Iniciais

- **`WEBHOOK_DS_AI_BUSINESS_STG`**: Define o ambiente para o webhook do Sentinel
  ```python
  WEBHOOK_DS_AI_BUSINESS_STG = 'stg'  # Ambiente de homologação/staging
  ```

### Função `error_message`

A função `error_message` é responsável pelo tratamento padronizado de erros durante a execução do pipeline, especialmente durante as operações de salvamento de dados.

#### Parâmetros
- `e` (Exception): A exceção capturada durante a execução

#### Comportamento
1. **Captura detalhada do erro**: 
   - Obtém o traceback completo usando `traceback.format_exc()`
   - Formata uma mensagem informativa com esses detalhes

2. **Envio de alerta**: 
   - Inicializa o objeto `Sentinel` com informações específicas do projeto:
     - `project_name`: 'Monitor_Linhas_Cuidado_Mama'
     - `env_type`: Ambiente definido em `WEBHOOK_DS_AI_BUSINESS_STG`
     - `task_title`: 'Pardini Mama'
   - Envia o alerta através de `alerta_sentinela()` com:
     - Categoria: 'Alerta'
     - Mensagem detalhada do erro
     - ID descritivo: '1_pardini_mama_birads'

3. **Registro do erro**:
   - Exibe o traceback completo no console via `traceback.print_exc()`
   - Relança a exceção para interromper a execução em caso de falha crítica

### Importância

Este sistema de tratamento de erros garante que:
- Problemas durante o processamento sejam detectados rapidamente
- A equipe responsável seja notificada automaticamente
- Informações detalhadas sobre o erro estejam disponíveis para diagnóstico
- O processo seja interrompido de forma apropriada em caso de falhas críticas

In [0]:
WEBHOOK_DS_AI_BUSINESS_STG = 'stg'

def error_message(e):
    """
    Envia alerta para o Sentinel e exibe o traceback em caso de erro ao salvar dados.

    Parâmetros:
        e (Exception): Exceção capturada.

    Comportamento:
        - Formata o traceback do erro.
        - Envia alerta para o Sentinel com detalhes do erro.
        - Exibe o traceback no console.
        - Relança a exceção.
    """
    error_message = traceback.format_exc()
    summary_message = f"""Erro ao salvar dados.\n{error_message}"""
    sentinela_ds_ai_business = Sentinel(
        project_name='Monitor_Linhas_Cuidado_Mama',
        env_type=WEBHOOK_DS_AI_BUSINESS_STG,
        task_title='Pardini Mama'
    )
    sentinela_ds_ai_business.alerta_sentinela(
        categoria='Alerta', 
        mensagem=summary_message,
        job_id_descritivo='1_pardini_mama_birads'
    )
    traceback.print_exc()
    raise e

## Funções de Persistência de Dados

Esta célula define três funções essenciais para gerenciar a persistência dos dados processados em tabelas Delta Lake: `insert_data`, `merge_data` e `save_data`.

### 1. Função `insert_data`

**Objetivo:**
Inserir dados em uma tabela Delta, realizando uma operação de sobrescrita completa (overwrite).

**Parâmetros:**
- `df_spk` (DataFrame): DataFrame Spark com os dados a serem inseridos
- `table_name` (str): Nome da tabela Delta de destino

**Comportamento:**
- Registra a operação no log (`logger.info`)
- Configura as opções de escrita para:
  - Modo 'overwrite' para substituição completa
  - Permitir alterações de schema (`mergeSchema=true`, `overwriteSchema=true`)
  - Usar o formato Delta Lake
  - Particionar os dados pela coluna `_datestamp` para otimizar consultas por data
- Captura exceções e aciona o tratamento de erros via `error_message()`

### 2. Função `merge_data`

**Objetivo:**
Realizar uma operação de merge (upsert) em tabelas Delta existentes, atualizando registros existentes e inserindo novos.

**Parâmetros:**
- `df_spk` (DataFrame): DataFrame Spark com os dados a serem mesclados
- `table_name` (str): Nome da tabela Delta de destino

**Comportamento:**
- Registra a operação no log
- Cria uma view temporária com os dados do DataFrame (`increment_birads`)
- Executa um comando MERGE SQL que:
  - Atualiza registros existentes (identificados pela chave composta `ficha`, `sequencial`, `sigla_exame`)
  - Insere novos registros quando não existem correspondências
- Captura exceções e aciona o tratamento de erros

### 3. Função `save_data`

**Objetivo:**
Função de alto nível que decide entre inserção ou mesclagem com base no contexto.

**Parâmetros:**
- `df_spk` (DataFrame): DataFrame Spark com os dados a serem salvos
- `table_name` (str): Nome da tabela Delta de destino

**Lógica:**
- Verifica se o DataFrame está vazio - se estiver, retorna sem ação
- Verifica se a tabela já existe:
  - Se existir, chama `merge_data()` para realizar upsert
  - Se não existir, chama `insert_data()` para criar a tabela

### Considerações Técnicas

- A função `merge_data` utiliza o padrão MERGE INTO do SQL, otimizado para operações de upsert em Delta Lake
- A partição por `_datestamp` melhora a performance para consultas filtradas por data
- A opção `overwriteSchema=true` permite que o schema evolua com o tempo, adicionando novos campos quando necessário

In [0]:
def insert_data(df_spk: DataFrame, table_name: str):
    """
    Insere os dados do DataFrame na tabela Delta especificada, sobrescrevendo o conteúdo existente.
    
    Parâmetros:
        df_spk (DataFrame): DataFrame a ser salvo.
        table_name (str): Nome da tabela Delta de destino.
    """
    try:
        logger.info(f"Inserting Data: {table_name}")
        (
            df_spk.write
                .mode('overwrite')
                .option('mergeSchema','true')
                .option('overwriteSchema','true')
                .format('delta')
                .partitionBy('_datestamp')
                .saveAsTable(table_name)
        )
    except Exception as e:
        error_message(e)
 
def merge_data(df_spk: DataFrame, table_name: str):
    """
    Realiza merge dos dados do DataFrame na tabela Delta, atualizando registros existentes e inserindo novos.
    
    Parâmetros:
        df_spk (DataFrame): DataFrame com os dados incrementais.
        table_name (str): Nome da tabela Delta de destino.
    """
    try:
        logger.info(f"Merging Data: {table_name}")
        df_spk.createOrReplaceTempView(f"increment_birads")
        merge_query = f"""
            MERGE INTO {table_name} AS target
            USING increment_birads AS source
                ON target.ficha = source.ficha
                AND target.sequencial = source.sequencial
                AND target.sigla_exame = source.sigla_exame
            WHEN MATCHED THEN
                UPDATE SET *
            WHEN NOT MATCHED THEN
                INSERT *
        """
        spark.sql(merge_query)
    except Exception as e:
        error_message(e)
 
def save_data(df_spk: DataFrame, table_name: str):
    """
    Salva os dados do DataFrame na tabela Delta, utilizando merge se a tabela existir ou insert se não existir.
    
    Parâmetros:
        df_spk (DataFrame): DataFrame a ser salvo.
        table_name (str): Nome da tabela Delta de destino.
    """
    if df_spk.isEmpty():
        return None
 
    if spark.catalog.tableExists(table_name):
        merge_data(df_spk, table_name)
    else:
        insert_data(df_spk, table_name)

## Aplicação das Transformações nos DataFrames

Esta célula aplica as transformações definidas na função `transform_fields` aos dois DataFrames gerados anteriormente: `df_spk` e `df_spk_ativacao`.

### Objetivo da Célula

Enriquecer os DataFrames com informações calculadas e transformar os dados para análise e persistência, aplicando as mesmas transformações tanto ao conjunto completo de dados quanto ao subconjunto de pacientes elegíveis para ativação.

### Operações Realizadas

1. **Para o DataFrame principal `df_spk`**:
   - Adiciona a coluna `retorno_cliente` com o tempo recomendado em meses
   - Calcula a data prevista de retorno em `dth_previsao_retorno`
   - Converte campos de data para formato timestamp
   - Calcula o intervalo em dias até o retorno em `dias_ate_retorno`

2. **Para o DataFrame de ativação `df_spk_ativacao`**:
   - Aplica as mesmas transformações que no DataFrame principal
   - Os dados já estão pré-filtrados para incluir apenas pacientes elegíveis para notificações

### Dependências

- Função `transform_fields` definida anteriormente
- DataFrames `df_spk` e `df_spk_ativacao` gerados pela consulta SQL

### Impacto

Após a execução desta célula, ambos os DataFrames estarão preparados com todos os campos calculados necessários para:
- Análises sobre os prazos de retorno recomendados
- Identificação de pacientes com exames em atraso
- Geração de notificações com informações precisas sobre datas e prazos

In [0]:
df_spk = transform_fields(df_spk)
df_spk_ativacao = transform_fields(df_spk_ativacao)

## Deduplicação de Registros para Notificações

Esta célula realiza a deduplicação de registros no DataFrame de ativação (`df_spk_ativacao`), para garantir que cada paciente receba apenas uma notificação por ficha, independentemente da quantidade de exames realizados.

### Objetivo da Célula

Eliminar registros duplicados do DataFrame `df_spk_ativacao` para que cada ficha (que representa um atendimento) apareça apenas uma vez na lista de ativação, evitando o envio de múltiplas notificações para o mesmo paciente sobre o mesmo atendimento.

### Lógica Implementada

A célula utiliza o método `.dropDuplicates(['ficha'])` do PySpark DataFrame API para:

1. Identificar registros com o mesmo valor na coluna `ficha`
2. Manter apenas a primeira ocorrência de cada valor único de `ficha`
3. Descartar todas as ocorrências duplicadas subsequentes

### Contexto Importante

Na estrutura de dados do Pardini:
- Uma `ficha` corresponde a um atendimento ou visita do paciente
- Um atendimento pode conter vários exames, representados por diferentes valores de `id_exame`, `sigla_exame`, etc.

Sem esta deduplicação, o sistema poderia enviar múltiplas notificações para o mesmo paciente sobre o mesmo atendimento, apenas porque foram realizados vários procedimentos durante aquela visita. A deduplicação garante que a comunicação com o paciente seja apropriada e não excessiva.

In [0]:
# Excluir duplicados para considerar apenas a ficha na ativação e não os exames (itens). Assim vamos enviar apenas 
# 1 push por ficha
df_spk_ativacao = df_spk_ativacao.dropDuplicates(['ficha'])

## Contagem de Laudos Processados

Esta célula exibe a quantidade de registros em cada DataFrame após todas as transformações e filtragens. Este é um passo importante para monitorar o volume de dados processados e validar se os filtros estão funcionando como esperado.

### Objetivo da Célula

Contar e exibir o número de registros nos dois principais DataFrames do pipeline, oferecendo uma verificação visual do volume de dados que serão persistidos nas tabelas Delta.

### Saídas Exibidas

Duas métricas principais são apresentadas:

1. **`quantidade de laudos salvos na tabela`**: Número total de laudos de mamografia do Pardini com classificação BI-RADS extraída e processada
   - Este valor representa todos os laudos processados, independentemente da classificação BI-RADS

2. **`quantidade de laudos salvos na tabela de ativação`**: Número de registros únicos (por ficha) que atendem aos critérios para notificação
   - Este subconjunto inclui apenas pacientes com BI-RADS 1, 2 ou 3, do sexo feminino, entre 40-75 anos e sem retorno já registrado
   - Os registros também foram deduplicados por ficha

### Utilidade das Métricas

Estas contagens são úteis para:
- Verificar a eficácia dos filtros aplicados
- Estimar o volume de notificações que serão enviadas
- Detectar possíveis problemas de dados (valores muito baixos ou muito altos podem indicar anomalias)
- Acompanhar tendências de volume ao longo do tempo, quando o notebook é executado periodicamente

In [0]:
print('quantidade de laudos salvos na tabela',df_spk.count())
print('quantidade de laudos salvos na tabela de ativação', df_spk_ativacao.count())

## Salvamento dos Dados nas Tabelas Delta

Esta célula final executa a persistência dos dados processados nas tabelas Delta Lake, concluindo o pipeline de processamento. Os dados serão utilizados para análises posteriores e para o sistema de notificação de pacientes.

### Objetivo da Célula

Salvar os dois DataFrames processados nas suas respectivas tabelas Delta:

1. **Tabela Principal de BI-RADS**: 
   ```python
   save_data(df_spk, table_birads)
   ```
   - Armazena todos os laudos com classificação BI-RADS extraída e dados enriquecidos
   - Utiliza a função inteligente `save_data()` que decide entre operações de merge ou insert baseado na existência prévia da tabela

2. **Tabela de Ativação**: 
   ```python
   insert_data(df_spk_ativacao, table_birads_ativacao)
   ```
   - Armazena apenas os pacientes elegíveis para notificações
   - Utiliza `insert_data()` para substituir completamente os dados, garantindo que apenas pacientes atualmente elegíveis estejam presentes

### Tabelas de Destino

- **`refined.saude_preventiva.pardini_laudos_mama_birads`**: Tabela histórica contendo todos os laudos processados
- **`refined.saude_preventiva.pardini_laudos_mama_birads_ativacao`**: Tabela com apenas os pacientes elegíveis para notificação (BI-RADS 1-3)

### Comportamento das Funções

- **Para a tabela principal**: 
  - Se a tabela já existe: Realiza operação de merge (upsert)
  - Se a tabela não existe: Cria a tabela com os dados do DataFrame

- **Para a tabela de ativação**: 
  - Sempre realiza operação de substituição completa (overwrite)
  - Particiona os dados pela coluna `_datestamp`

### Impacto

Após a execução desta célula, os dados estarão disponíveis para consumo por outros sistemas, como dashboards de BI ou sistemas de notificação automática de pacientes. A tabela de ativação, em particular, será utilizada para gerar alertas para pacientes que necessitam realizar exames de acompanhamento.

In [0]:
save_data(df_spk, table_birads)
insert_data(df_spk_ativacao, table_birads_ativacao)